In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from pathlib import Path
import platform
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from matplotlib import dates, ticker

import oshe as tc

Failed to load paths from C:\Users\tgerrish\Miniconda3\lib\site-packages\honeybee\config.json.
Path to radiance is set to: c:/radiance


In [23]:
fp = "../ARE_AZ_Sharjah.Sw.404490_TMYx.epw"
df = tc.helpers.load_weather(fp)
out_dir = tc.helpers.create_directory("../plots")

In [30]:
# Create dirunal profile charts
moistures = ["rh", "wbt", "dpt"]
for m in moistures:
    tc.plot.diurnal(df, moisture=m, save_path=out_dir / "diurnal_{0:}.png".format(m))

Diurnal plot saved to ..\plots\diurnal_rh.png
Diurnal plot saved to ..\plots\diurnal_wbt.png
Diurnal plot saved to ..\plots\diurnal_dpt.png


In [31]:
# Create heatmaps
_vars = {
    "dry_bulb_temperature": "Reds",
    "global_horizontal_radiation": "Oranges",
    "wind_speed": "Greys",
    "relative_humidity": "Blues",
    "wet_bulb_temperature": "Greens",
    "total_sky_cover": "Purples",
}
for k, v in _vars.items():
    tc.plot.heatmap_generic(df[k], cmap=v, save_path=out_dir / "heatmap_{0:}.png".format(k))

Heatmap plot saved to ..\plots\heatmap_dry_bulb_temperature.png
Heatmap plot saved to ..\plots\heatmap_global_horizontal_radiation.png
Heatmap plot saved to ..\plots\heatmap_wind_speed.png
Heatmap plot saved to ..\plots\heatmap_relative_humidity.png
Heatmap plot saved to ..\plots\heatmap_wet_bulb_temperature.png
Heatmap plot saved to ..\plots\heatmap_total_sky_cover.png


In [84]:
# Create windroses
for s in ["Annual","Spring","Summer","Autumn","Winter","Shoulder"]:
    for d in ["Daily", "Morning", "Midday", "Afternoon", "Evening", "Night", "MorningShoulder", "AfternoonShoulder"]:
        tc.plot.windrose(df, season_period=s, day_period=d, n_sector=16, save_path=out_dir / "windrose_{0:}_{1:}.png".format(d, s), close=True)

C:\Users\tgerrish\Desktop\sharjahtemp\oshe\oshe\plot.py:1353: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Windrose saved to ..\plots\windrose_Daily_Annual.png
Windrose saved to ..\plots\windrose_Morning_Annual.png
Windrose saved to ..\plots\windrose_Midday_Annual.png
Windrose saved to ..\plots\windrose_Afternoon_Annual.png
Windrose saved to ..\plots\windrose_Evening_Annual.png
Windrose saved to ..\plots\windrose_Night_Annual.png
Windrose saved to ..\plots\windrose_MorningShoulder_Annual.png
Windrose saved to ..\plots\windrose_AfternoonShoulder_Annual.png
Windrose saved to ..\plots\windrose_Daily_Spring.png
Windrose saved to ..\plots\windrose_Morning_Spring.png
Windrose saved to ..\plots\windrose_Midday_Spring.png
Windrose saved to ..\plots\windrose_Afternoon_Spring.png
Windrose saved to ..\plots\windrose_Evening_Spring.png
Windrose saved to ..\plots\windrose_Night_Spring.png
Windrose saved to ..\plots\windrose_MorningShoulder_Spring.png
Windrose saved to ..\plots\windrose_AfternoonShoulder_Spring.png
Windrose saved to ..\plots\windrose_Daily_Summer.png
Windrose saved to ..\plots\windrose_M

<Figure size 432x432 with 0 Axes>

In [39]:
# Create wind frequency
for s in ["Annual","Spring","Summer","Autumn","Winter","Shoulder"][0:]:
    for d in ["Daily", "Morning", "Midday", "Afternoon", "Evening", "Night", "MorningShoulder", "AfternoonShoulder"][0:]:
        tc.plot.wind_frequency(df, season_period=s, day_period=d, save_path=out_dir / "windfreq_{0:}_{1:}.png".format(d, s), close=True)

Wind frequency histogram saved to ..\plots\windfreq_Daily_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_Morning_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_Midday_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_Afternoon_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_Evening_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_Night_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_MorningShoulder_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_AfternoonShoulder_Annual.png
Wind frequency histogram saved to ..\plots\windfreq_Daily_Spring.png
Wind frequency histogram saved to ..\plots\windfreq_Morning_Spring.png
Wind frequency histogram saved to ..\plots\windfreq_Midday_Spring.png
Wind frequency histogram saved to ..\plots\windfreq_Afternoon_Spring.png
Wind frequency histogram saved to ..\plots\windfreq_Evening_Spring.png
Wind frequency histogram saved to ..\plots\windfreq_Night_Sprin

In [52]:
# Open field analysis
idd_file = r"C:\openstudio-2.9.0\EnergyPlus\Energy+.idd"

def generate_results(epw_file, idd_file, ground_material, shaded=False):
    
    # Load epw
    epw = tc.helpers.load_weather(epw_file)

    # Get climate variables
    dbt = epw.dbt
    rh = epw.rh
    ws = epw.ws
    hir = epw.hir
    sun_altitude = epw.sun_altitude
    
    # Define ground zone from material (using predefined dictionary of properties)
    gnd_mat = tc.material.material_dict[ground_material]
    ground_zone = tc.geometry.Ground(gnd_mat, xy=5, depth=1.5, subsurface_size=5)
    
    # Create shade objects if shaded
    if shaded:
        shades = [
            tc.geometry.Shade(vertices=[[5, 5, 3], [5, 5, 0], [5, -5, 0], [5, -5, 3]]),
            tc.geometry.Shade(vertices=[[5, -5, 3], [5, -5, 0], [-5, -5, 0], [-5, -5, 3]]),
            tc.geometry.Shade(vertices=[[-5, -5, 3], [-5, -5, 0], [-5, 5, 0], [-5, 5, 3]]),
            tc.geometry.Shade(vertices=[[-5, 5, 3], [-5, 5, 0], [5, 5, 0], [5, 5, 3]]),
            tc.geometry.Shade(vertices=[[-5, -5, 3], [-5, 5, 3], [5, 5, 3], [5, -5, 3]]),
        ]
    else:
        shades = None
    
    of_srf_temp = tc.energyplus.run_energyplus(epw_file, idd_file, ground=ground_zone, shades=shades, run=True).to_frame().values.T[0]  # Calculate ground surface temperature
    of_dir_rad, of_dif_rad = tc.radiance.run_radiance(epw_file, ground=ground_zone, shades=shades, run=True)  # Calculate incident solar direct and diffuse radiation
    
    # Calculate MRT & UTCI
    of_mrt = tc.mrt.mean_radiant_temperature(surrounding_surfaces_temperature=of_srf_temp[0], horizontal_infrared_radiation_intensity=hir, diffuse_horizontal_solar=of_dif_rad.T[0], direct_normal_solar=of_dir_rad.T[0], sun_altitude=sun_altitude, ground_reflectivity=gnd_mat.reflectivity, sky_exposure=0 if shaded else 1)[0]
    of_utci = tc.utci.universal_thermal_climate_index(dbt, of_mrt, ws, rh)
    
    return {
        ground_material + "_{0:}".format("SHADED" if shaded else "UNSHADED"): {
            "srf_temp": of_srf_temp[0],
            "mrt": of_mrt,
            "utci": of_utci
        }
    }

# Simulate cases
res = []
for mat in ["CONCRETE", "GRASS"]:
    for shaded in [True, False]:
        print("\nRunning for {} {}".format("SHADED" if shaded else "UNSHADED", mat))
        res.append(generate_results(fp, idd_file, mat, shaded=shaded))

# Join results into DataFrame
result = {}
for d in res:
    result.update(d)

ll = []
for k, v in result.items():
    ll.append(pd.DataFrame.from_dict(v).set_index(tc.helpers.ANNUAL_DATETIME))

comf_df = pd.concat(ll, axis=1, keys=list(result.keys()))


Running for SHADED CONCRETE
Simulation completed
Writing recipe contents to: C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual
Calculating solar values...
# Number of sun up hours: 4428
Writing sun matrix to C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual\sky\sunmtx.smx
Direct and diffuse solar radiation simulation completed
Radiance results loaded
Mean radiant temperature calculated [0.06 seconds]


C:\Users\tgerrish\Desktop\sharjahtemp\oshe\oshe\utci.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_speed[wind_speed < 0.5] = 0.5
C:\Users\tgerrish\Desktop\sharjahtemp\oshe\oshe\utci.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_speed[wind_speed > 17] = 17


Universal thermal climate index calculated [0.19 seconds]

Running for UNSHADED CONCRETE
Simulation completed
Writing recipe contents to: C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual
Calculating solar values...
# Number of sun up hours: 4428
Writing sun matrix to C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual\sky\sunmtx.smx
Direct and diffuse solar radiation simulation completed
Radiance results loaded
Mean radiant temperature calculated [0.09 seconds]
Universal thermal climate index calculated [0.20 seconds]

Running for SHADED GRASS
Simulation completed
Writing recipe contents to: C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual
Calculating solar values...
# Number of sun up hours: 4428
Writing sun matrix to C:\Users\tgerrish\AppData\Local\Temp\openfield\gridbased_annual\sky\sunmtx.smx
Direct and diffuse solar radiation simulation completed
Radiance results loaded
Mean radiant temperature calculated [0.06 seconds]
Universal thermal c

In [83]:
ums = ["CONCRETE_SHADED", "CONCRETE_UNSHADED", "GRASS_SHADED", "GRASS_UNSHADED"]
for um in ums[0:]:
    title = "{} ({})\n{} - {} - {}".format(um.split("_")[0].title(), um.split("_")[1].title(), df.city.values[0], df.country.values[0], df.station_id.values[0])
    tc.plot.utci_comfort_heatmap(comf_df[um]["utci"].values, tone_color="k", invert_y=False, title=title, save_path=out_dir / "utciheatmap_{0:}_{1:}.png".format(um.split("_")[0].lower(), um.split("_")[1].lower()), close=True)
    
    tc.plot.utci_heatmap_detailed_generic(comf_df[um]["utci"], tone_color="k", invert_y=False, title=title, save_path=out_dir / "utciheatmapdetailed_{0:}_{1:}.png".format(um.split("_")[0].lower(), um.split("_")[1].lower()), close=True)

Plot saved to ..\plots\utciheatmap_concrete_shaded.png
UTCI detailed saved to ..\plots\utciheatmapdetailed_concrete_shaded.png
Plot saved to ..\plots\utciheatmap_concrete_unshaded.png
UTCI detailed saved to ..\plots\utciheatmapdetailed_concrete_unshaded.png
Plot saved to ..\plots\utciheatmap_grass_shaded.png
UTCI detailed saved to ..\plots\utciheatmapdetailed_grass_shaded.png
Plot saved to ..\plots\utciheatmap_grass_unshaded.png
UTCI detailed saved to ..\plots\utciheatmapdetailed_grass_unshaded.png
